# From CM to trial results

## Introduction

Goal of this cookbook is to illustrate how one can post the material needed for a trial, run it and visualize the results.

Steps: 
1. Post a Computational Model
2. Create a Virtual Population (Vpop) Design
3. Generate a Vpop from the Vpop Design
4. Post a Protocol
5. Post a Data Table
6. Post a Trial
7. Run and monitor a Trial
8. Visualize the trial results

Linked resources: 
- [Folder on jinko](https://jinko.ai/project/e0fbb5bb-8929-439a-bad6-9e12d19d9ae4).

In [1]:
%pip install jinko-sdk

# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Authentication successful


In [2]:
# Cookbook specifics imports

%pip install plotly
import io
import json
import os
import pandas as pd
import plotly.express as px
import zipfile

# Cookbook specifics constants:
# put here the constants that are specific to your cookbook like
# the reference to the Jinko items, the name of the model, etc.

# @param {"name":"folderId", "type": "string"}
# folder_id can be retrieved in the url, pattern is `https://jinko.ai/project/<project_id>?labels=<folder_id>`
folder_id = "a21d004f-a1f9-4e79-a8dd-a38bba74c4a3"

resources_dir = os.path.normpath("resources/run_a_trial")

model_file = os.path.join(resources_dir, "computational_model.json")
model_file_copy = os.path.join(resources_dir, "computational_model_copy.json")

solving_options_file = os.path.join(resources_dir, "solving_options.json")
vpop_file = os.path.join(resources_dir, "vpop.csv")
protocol_file = os.path.join(resources_dir, "protocol.json")
data_table_file = os.path.join(resources_dir, "data_table.csv")

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Step 1: Post a Computational Model

In [3]:
# Load the model
with open(model_file, "r") as f:
    model = json.load(f)

# Load the solving options
with open(solving_options_file, "r") as f:
    solving_options = json.load(f)

# Post the model with its options
# https://doc.jinko.ai/api/#/paths/core-v2-model_manager-jinko_model/post
response = jinko.make_request(
    path="/core/v2/model_manager/jinko_model",
    method="POST",
    json={"model": model, "solvingOptions": solving_options},
    options={"name": "simple tumor model", "folder_id": folder_id},
)

# Get the ids of the resource
project_item_info = jinko.get_project_item_info_from_response(response)
model_core_item_id = project_item_info["coreItemId"]["id"]
model_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

# Get the URL of the resource
print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/cm-u3yL-eKz8


201: {"coreItemId":"e5daa806-63f5-4142-912f-fb66711876d2","metadata":{"internal":{"contentHash":"ad612f371b1c60de1e678376fd7819a84d67ebe2127ddce3f1fede76894731be","projectId":"e0fbb5bb-8929-439a-bad6-9e12d19d9ae4"},"public":{"algebraicRuleCount":0,"baselineCheckCount":0,"categoricalParameterCount":1,"compartmentCount":5,"eventCount":2,"odeCount":0,"parameterCount":25,"reactionCount":11,"speciesCount":6}},"snapshotId":"d4cb15e2-dae2-4c0b-9a12-aeddb44aea5c","warnings":[]}


# Step 2: Create a Vpop Design

### Get model descriptors


In [4]:
# get model baseline descriptors
# https://doc.jinko.ai/api/#/paths/core-v2-model_manager-jinko_model-jinkoModelId--snapshots--jinkoModelIdSnapshot--baseline_descriptors/get
response = jinko.make_request(
    path=f"/core/v2/model_manager/jinko_model/{model_core_item_id}/snapshots/{model_snapshot_id}/baseline_descriptors",
)

response_json = response.json()

numeric_descriptors = response_json["numericDescriptors"]

# build the default marginal distributions that will be used to create the vpop design
# we only select descriptors that are PatientDescriptorKnown, PatientDescriptorUnknown or PatientDescriptorPartiallyKnown
default_marginal_distributions = [
    {
        "distribution": {
            "highBound": descriptor["distribution"]["highBound"],
            "lowBound": descriptor["distribution"]["lowBound"],
            "tag": descriptor["distribution"]["tag"],
        },
        "id": descriptor["id"],
    }
    for descriptor in numeric_descriptors
    if any(
        tag in descriptor["inputTag"]
        for tag in [
            "PatientDescriptorKnown",
            "PatientDescriptorUnknown",
            "PatientDescriptorPartiallyKnown",
        ]
    )
]

# Creating a formatted message with the IDs from marginal distributions
ids_output = "IDs present in the Marginal Distributions:\n" + "\n".join(
    [distribution["id"] for distribution in default_marginal_distributions]
)

default_marginal_distributions

[{'distribution': {'highBound': 1.2, 'lowBound': 0.8, 'tag': 'Uniform'},
  'id': 'Blood'},
 {'distribution': {'highBound': 0.36, 'lowBound': 0.24, 'tag': 'Uniform'},
  'id': 'Lymph'},
 {'distribution': {'highBound': 12, 'lowBound': 8, 'tag': 'Uniform'},
  'id': 'SubcutaneousInjectionSite'},
 {'distribution': {'highBound': 0.6, 'lowBound': 0.4, 'tag': 'Uniform'},
  'id': 'Tissue'},
 {'distribution': {'highBound': 120, 'lowBound': 80, 'tag': 'Uniform'},
  'id': 'bloodFlowRate'},
 {'distribution': {'highBound': 0.0006, 'lowBound': 0.0004, 'tag': 'Uniform'},
  'id': 'ec50Drug'},
 {'distribution': {'highBound': 60, 'lowBound': 40, 'tag': 'Uniform'},
  'id': 'initialTumorBurden'},
 {'distribution': {'highBound': 0.096, 'lowBound': 0.064, 'tag': 'Uniform'},
  'id': 'kClearanceDrug'},
 {'distribution': {'highBound': 0.0012, 'lowBound': 0.0008, 'tag': 'Uniform'},
  'id': 'kGrowthCancerCell'},
 {'distribution': {'highBound': 1200000000000,
   'lowBound': 800000000000,
   'tag': 'Uniform'},
  'id

### Create a new list with the updated distributions

In [5]:
# Define a dictionary for distribution settings
distribution_settings = {
    "initialTumorBurden": {"mean": 1.8, "stdev": 0.08, "base": 10, "tag": "LogNormal"},
    "kccCancerCell": {"mean": 12, "stdev": 0.5, "base": 10, "tag": "LogNormal"},
    "kGrowthCancerCell": {"mean": -3, "stdev": 0.05, "base": 10, "tag": "LogNormal"},
    "vmaxCancerCellDeath": {"mean": -1, "stdev": 0.05, "base": 10, "tag": "LogNormal"},
    "ec50Drug": {"mean": -3.5, "stdev": 0.05, "base": 10, "tag": "LogNormal"},
}

# Refactor the list comprehension using the dictionary
updated_marginal_distributions = [
    {
        "id": element["id"],
        "distribution": distribution_settings.get(
            element["id"],
            element[
                "distribution"
            ],  # Default to the existing distribution if id is not found
        ),
    }
    for element in default_marginal_distributions
]

### Post the vpop design

In [6]:
# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop_generator/post
response = jinko.make_request(
    path="/core/v2/vpop_manager/vpop_generator",
    method="POST",
    json={
        "contents": {
            "computationalModelId": {
                "coreItemId": model_core_item_id,
                "snapshotId": model_snapshot_id,
            },
            "correlations": [],
            "marginalCategoricals": [],
            "marginalDistributions": updated_marginal_distributions,
        },
        "tag": "VpopGeneratorFromDesign",
    },
    options={
        "name": "vpop design for simple tumor model",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
vpop_generator_core_item_id = project_item_info["coreItemId"]["id"]
vpop_generator_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/vd-3lwq-veNt


201: {"coreItemId":"02ff57c5-fc8e-425d-8861-b1c53f8f4fb5","metadata":{"internal":{"tag":"VpopGeneratorMetadataEmpty"},"public":{"correlationCount":0,"descriptorCount":12}},"snapshotId":"2e1b78dc-3076-47b7-a57d-58e75da98eb8","warnings":[]}


# Step 3: Generate a Vpop from the Vpop design

In [7]:
# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop_generator-vpopGeneratorId--snapshots--vpopGeneratorIdSnapshot--vpop/post

response = jinko.make_request(
    path=f"/core/v2/vpop_manager/vpop_generator/{vpop_generator_core_item_id}/snapshots/{vpop_generator_snapshot_id}/vpop",
    method="POST",
    json={
        "contents": {
            "computationalModelId": {
                "coreItemId": model_core_item_id,
                "snapshotId": model_snapshot_id,
            },
            "size": 10,  # vpop has 10 patients
        },
        "tag": "VpopGeneratorOptionsForVpopDesign",
    },
    options={
        "name": "vpop for simple tumor model",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
vpop_core_item_id = project_item_info["coreItemId"]["id"]
vpop_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/vp-aehA-UWuR


# Step 3 bis - not mandatory: Directly post a csv vpop

In [8]:
with open(vpop_file, "r") as file:
    vpop = file.read()

# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop/post

response = jinko.make_request(
    path=f"/core/v2/vpop_manager/vpop",
    method="POST",
    csv_data=vpop,
    options={
        "name": "vpop for simple tumor model",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
vpop_bis_core_item_id = project_item_info["coreItemId"]["id"]
vpop_bis_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/vp-0HGm-Xv0F


201: {"coreItemId":"272fd0f2-0b1b-4354-b133-139d6b796911","metadata":{"internal":{"contentHash":"b9f309415d7ef7c0053ccb87c433e5563c6d7e331eff1cac215370747ff0e4dd","projectId":"e0fbb5bb-8929-439a-bad6-9e12d19d9ae4"},"public":{"descriptorCount":12,"patientCount":10}},"snapshotId":"f2c264a1-511e-4da2-a916-a0f818778cd1","warnings":[]}


# Step 4 : Post a Protocol

In [9]:
# Load the protocol
with open(protocol_file, "r") as f:
    protocol = json.load(f)

# Post the protocol
# https://doc.jinko.ai/api/#/paths/core-v2-scenario_manager-protocol_design/post
response = jinko.make_request(
    path="/core/v2/scenario_manager/protocol_design",
    method="POST",
    json=protocol,
    options={
        "name": "protocol for simple tumor model",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
protocol_core_item_id = project_item_info["coreItemId"]["id"]
protocol_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/pd-5bpe-mA7y


201: {"coreItemId":"7d23096f-cf7c-4b03-8d50-8328ca0bbd23","metadata":{"internal":{"contentHash":"5a72f106e63d15986b34838f6a652bf8b5395258e261c7fa9b5c11f7b1b9d888","projectId":"e0fbb5bb-8929-439a-bad6-9e12d19d9ae4"},"public":{"activeArmCount":12,"armCount":12,"armOverrideCount":3}},"snapshotId":"930d717c-b1dc-4a9e-8d52-5c5c2e91abf5","warnings":[]}


# Step 5: Post a Data Table

In [10]:
# the source data table is a CSV file, we convert it to SQLite and encode it in base64

encoded_data_table = jinko.data_table_to_sqlite(data_table_file)

# Step 3: Post the data table
# https://doc.jinko.ai/api/#/paths/core-v2-data_table_manager-data_table/post
response = jinko.make_request(
    path="/core/v2/data_table_manager/data_table",
    method="POST",
    json={
        "mappings": [],
        "rawData": encoded_data_table,
    },
    options={
        "name": "data table for simple tumor model",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
data_table_core_item_id = project_item_info["coreItemId"]["id"]
data_table_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/dt-FVlA-EzKY


201: {"coreItemId":"be6d7299-8355-42f5-baa8-cffd99cdbe83","metadata":{"internal":{"contentHash":"e88fb7928e8be37afbd83161d4d95538b8c9443eda3fb9228a916434841f49c2","projectId":"e0fbb5bb-8929-439a-bad6-9e12d19d9ae4"},"public":{"initialColumnCount":4,"initialRowCount":60,"observableCount":1,"timePointCount":5,"validForFitnessFunction":true,"withMappingColumnCount":4,"withMappingRowCount":60}},"snapshotId":"214dafb2-b66e-416d-b276-0f8b55cff7c8","warnings":[]}


# Step 6: Post a Trial

In [11]:
# https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial/post

# Define the data payload
trial_data = {
    "computationalModelId": {
        "coreItemId": model_core_item_id,
        "snapshotId": model_snapshot_id,
    },
    "protocolDesignId": {
        "coreItemId": protocol_core_item_id,
        "snapshotId": protocol_snapshot_id,
    },
    "vpopId": {"coreItemId": vpop_core_item_id, "snapshotId": vpop_snapshot_id},
    "dataTableDesigns": [
        {
            "dataTableId": {
                "coreItemId": data_table_core_item_id,
                "snapshotId": data_table_snapshot_id,
            },
            "options": {
                "logTransformWideBounds": [],
                "label": "data_table_simple_tumor",
            },
            "include": True,
        }
    ],
}

response = jinko.make_request(
    path="/core/v2/trial_manager/trial",
    method="POST",
    json=trial_data,
    options={
        "name": "trial for simple tumor model",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
trial_core_item_id = project_item_info["coreItemId"]["id"]
trial_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")

Resource link: https://jinko.ai/tr-NgNl-W0TW


201: {"coreItemId":"a0a81067-9093-49b0-9398-e2d3f7f3d0df","metadata":{"internal":{"contentHash":"c834c7214f043fe82938a84d09ddb203ab733b852e9b7e924f4d12283b7dc556","solveScoreVpopUuid":null,"status":"not_launched"},"public":{"armCount":12,"patientCount":10}},"snapshotId":"e9c95937-bf0e-4253-9eeb-dddaa5b1eb43","warnings":[]}


# Step 7 : Run and monitor a trial


### Run the trial

In [12]:
# https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-trialId--snapshots--trialIdSnapshot--run/post
response = jinko.make_request(
    path=f"/core/v2/trial_manager/trial/{trial_core_item_id}/snapshots/{trial_snapshot_id}/run",
    method="POST",
)

Query successfully done, got a 204 response


### get trial status


In [13]:
jinko.monitor_trial_until_completion(trial_core_item_id, trial_snapshot_id)

Trial Progress: 100%|██████████| 120/120 [00:11<00:00, 10.46tasks/s]

Job succeeded.


,Arm,countError,countPending,countSuccess
0,iv-0.1-10,0,0,10
1,iv-0.1-30,0,0,10
2,iv-1-10,0,0,10
3,iv-1-30,0,0,10
4,iv-5-10,0,0,10
5,iv-5-30,0,0,10
6,sc-0.1-10,0,0,10
7,sc-0.1-30,0,0,10
8,sc-1-10,0,0,10
9,sc-1-30,0,0,10


# Step 8 : Visualize the trial results 

In [14]:
# Retrieve time series ids (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-trialId--snapshots--trialIdSnapshot--output_ids/get)

response = jinko.make_request(
    "/core/v2/trial_manager/trial/%s/snapshots/%s/output_ids"
    % (trial_core_item_id, trial_snapshot_id),
    method="GET",
)
responseSummary = json.loads(response.content.decode("utf-8"))
print("Available time series:\n", responseSummary, "\n")

Available time series:
 [{'displayName': None, 'id': 'Time'}, {'displayName': 'Memory usage', 'id': '__jinkoAllocationMiB'}, {'displayName': 'Solver re-initialisation count', 'id': '__jinkoNumReInit'}, {'displayName': 'Solver steps count', 'id': '__jinkoNumSteps'}, {'displayName': 'Rhs function calls', 'id': '__jinkoRhsCalls'}, {'displayName': 'Root function calls', 'id': '__jinkoRootFunctionCalls'}, {'displayName': 'Solving time', 'id': '__jinkoSolvingTime'}, {'displayName': None, 'id': 'tumorBurden'}] 



In [15]:
# Retrieve time series (https://doc.jinko.ai/api/#/paths/core-v2-result_manager-timeseries_summary/post)

# replace here by the time series ids list you want
idsForTimeSeries = ["tumorBurden"]

try:
    print("Retrieving time series data...")
    response = jinko.make_request(
        "/core/v2/result_manager/timeseries_summary",
        method="POST",
        json={
            "select": idsForTimeSeries,
            "trialId": {
                "coreItemId": trial_core_item_id,
                "snapshotId": trial_snapshot_id,
            },
        },
    )
    if response.status_code == 200:
        print("Time series data retrieved successfully.")
        archive = zipfile.ZipFile(io.BytesIO(response.content))
        filename = archive.namelist()[0]
        print(f"Extracted time series file: {filename}")
        csvTimeSeries = archive.read(filename).decode("utf-8")
    else:
        print(
            f"Failed to retrieve time series data: {response.status_code} - {response.reason}"
        )
        response.raise_for_status()
except Exception as e:
    print(f"Error during time series retrieval or processing: {e}")
    raise

Retrieving time series data...
Time series data retrieved successfully.
Extracted time series file: timeseries-f3230453-4da7-5169-b5a2-95f55a81a358.csv


In [16]:
dfTimeSeries = pd.read_csv(io.StringIO(csvTimeSeries))
display(dfTimeSeries.head(5))

# Extract unique patient IDs
unique_patient_ids = dfTimeSeries["Patient Id"].unique().tolist()

# Display unique patient IDs
print(unique_patient_ids)

,Patient Id,Arm,Descriptor,Time,Value
0,0a33e347-c3d0-efc2-c11b-d546c92c7f26,iv-0.1-10,tumorBurden,0.0,45.701616
1,0a33e347-c3d0-efc2-c11b-d546c92c7f26,iv-0.1-10,tumorBurden,21600.0,45.709608
2,0a33e347-c3d0-efc2-c11b-d546c92c7f26,iv-0.1-10,tumorBurden,43200.0,45.720472
3,0a33e347-c3d0-efc2-c11b-d546c92c7f26,iv-0.1-10,tumorBurden,64800.0,45.732555
4,0a33e347-c3d0-efc2-c11b-d546c92c7f26,iv-0.1-10,tumorBurden,86400.0,45.744879


['0a33e347-c3d0-efc2-c11b-d546c92c7f26', '1a94e14a-ce73-5a5b-72e3-e7eb3b89ae95', '1dcf0a0a-27ee-6783-2434-d5fa813d8981', '21411afc-7e38-8a80-8cb0-6e7bcfc0f524', '46408c20-c0d8-55c2-a620-8ac6019b6d4a', '51ce2695-a89b-05a0-b361-52cc94dcbcda', '75e959fc-9601-c1ca-138d-eb8b78fb9c0e', '82e3f080-7207-03db-ab7e-4b0b07df2f30', 'd962ed3c-98dc-7da3-34dc-0769e64061ff', 'e2a1d7a3-8c3f-227a-d431-5c85a0b87bf6']


In [17]:
%pip install nbformat
# Filter data for the first patient
patient_data = dfTimeSeries[dfTimeSeries["Patient Id"] == unique_patient_ids[0]]

# Plot using Plotly
fig = px.line(
    patient_data,
    x="Time",
    y="Value",
    color="Arm",
    title="Time Series of Tumor Burden",
    labels={"Time": "Time (seconds)", "Value": "Tumor Burden Value"},
    markers=True,
)

jinko.show_plot_conditionally(fig)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
